# RFMIP-CLEAR-SKY example

*Last edited: 2024-10-17*

Compute clear-sky longwave fluxes.

Based on:

Ukkonen, P., & Hogan, R. J. (2023). Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0. Geoscientific Model Development, 16(11), 3241–3261. <https://doi.org/10.5194/gmd-16-3241-2023>. Code: <https://github.com/peterukk/rte-rrtmgp-nn>

Instructions for building are at: `examples/rfmip-clear-sky/Readme.md`

## Notes

- From the Readme.md: "The example programs have been modified to allow neural networks to be used by setting `use_rrtmgp_nn = .true.` (was `use_nn = .true.`) in `rrttmgp_rfmip_{lw|sw}.F90`. In addition, there is an option (`compare_flux =.true.`) to compare the output fluxes to benchmark line-by-line computations, alongside reference RRTMGP results which were produced in double precision."

    - When calling `rrtmgp_rfmip_{lw|sw}`, if the number of parameters is greater than or equal to 6, then the `use_rrtmgp_nn` flag is set to .true.

## Build the RTE+RRTMGP libraries

In [3]:
%cd /home/x/git/radnn/ukk23test01/build

/home/x/git/radnn/ukk23test01/build


In [4]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -m64 -march=native -O3
%env NCHOME=/usr
%env NFHOME=/usr
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -m64 -march=native -O3
env: NCHOME=/usr
env: NFHOME=/usr
env: BLASLIB=openblas


In [5]:
! make clean

rm -f *.optrpt *.mod *.o librrtmgp.a librte.a libneural.a


In [6]:
! make

gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_kind.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_rrtmgp_config.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_util_array.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_optical_props_kernels.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_optical_props.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_source_functions.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_fluxes_broadband_kernels.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_fluxes.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_rte_solver_kernels.F90
gfortran -ffree-line-length-none -m64 

## Build the examples

In [7]:
%cd /home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky

/home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky


In [10]:
! make clean

VAR="../../"
rm rrtmgp_rfmip_sw rrtmgp_rfmip_lw *.o *.mod *.optrpt
rm: cannot remove 'rrtmgp_rfmip_sw': No such file or directory
rm: cannot remove 'rrtmgp_rfmip_lw': No such file or directory
rm: cannot remove '*.optrpt': No such file or directory
make: [Makefile:142: clean] Error 1 (ignored)


In [11]:
! make

VAR="../../"
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c ../mo_simple_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c mo_rfmip_io.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c ../mo_load_coefficients.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c rrtmgp_rfmip_lw.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -o rrtmgp_rfmip_lw rrtmgp_rfmip_lw.o mo_simple_netcdf.o mo_rfmip_io.o mo_load_coefficients.o ../..//build/librte.a ../..//build/librrtmgp.a ../..//build/libneural.a -Wl,-O2 -Wl,--sort-common -Wl,--as-needed -Wl,-z,relro -Wl,-z,now -Wl,--disable-new-dtags -Wl,--gc-sections -Wl,--allow-shlib-undefined -Wl,-rpath,/home/x/conda/envs/tf2/lib -Wl,-rpath-link,/home/x/conda/envs/tf2/lib -L/home/x/conda/envs/tf2/lib  -L/home/x/conda/envs/tf2/targets/x86_64

## Data

In [1]:
import netCDF4 as nc

- The NC file, also known as the NetCDF file, stands for Network Common Data Form. It is a file format commonly used to store multidimensional scientific data. The following cells show the names of the variables contained in the .nc files [[Source](https://stackoverflow.com/questions/31931483/programmatically-list-all-variables-of-a-netcdf-file-using-netcdf4-and-python)]:
- To list all variable names as strings just do: list(<dset>.variables)
    - "variable names" are the unique identifiers given to each variable stored in the file
    - Cada variável representa um array multidimensional de dados e é definida por um nome e um ID
        - For example, in a "nc" file that stores weather data, you might have variables like temperature, humidity, pressure, each with their own data arrays

The softlink "/home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky/data/" points to "/home/x/data/7413952-ukk23-code-data/examples/rfmip-clear-sky/data"

In [13]:
PATH = "/home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky/data/"
FILE = "multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
print(list(nc.Dataset(PATH+FILE).variables))

['c2f6_GM', 'c3f8_GM', 'c4f10_GM', 'c5f12_GM', 'c6f14_GM', 'c7f16_GM', 'c8f18_GM', 'c_c4f8_GM', 'carbon_dioxide_GM', 'carbon_monoxide_GM', 'carbon_tetrachloride_GM', 'cf4_GM', 'cfc113_GM', 'cfc114_GM', 'cfc115_GM', 'cfc11_GM', 'cfc11eq_GM', 'cfc12_GM', 'cfc12eq_GM', 'ch2cl2_GM', 'ch3ccl3_GM', 'chcl3_GM', 'expt_label', 'halon1211_GM', 'halon1301_GM', 'halon2402_GM', 'hcfc141b_GM', 'hcfc142b_GM', 'hcfc22_GM', 'hfc125_GM', 'hfc134a_GM', 'hfc134aeq_GM', 'hfc143a_GM', 'hfc152a_GM', 'hfc227ea_GM', 'hfc236fa_GM', 'hfc23_GM', 'hfc245fa_GM', 'hfc32_GM', 'hfc365mfc_GM', 'hfc4310mee_GM', 'lat', 'lon', 'methane_GM', 'methyl_bromide_GM', 'methyl_chloride_GM', 'nf3_GM', 'nitrogen_GM', 'nitrous_oxide_GM', 'oxygen_GM', 'ozone', 'pres_layer', 'pres_level', 'profile_weight', 'sf6_GM', 'so2f2_GM', 'solar_zenith_angle', 'sst', 'surface_albedo', 'surface_emissivity', 'surface_temperature', 'temp_layer', 'temp_level', 'time', 'total_solar_irradiance', 'water_vapor']


In [14]:
PATH = "/home/x/git/radnn/ukk23test01/rrtmgp/data/"
FILE = "rrtmgp-data-lw-g256-2018-12-04.nc"
print(list(nc.Dataset(PATH+FILE).variables))

['key_species', 'kmajor', 'plank_fraction', 'temperature_Planck', 'totplnk', 'absorption_coefficient_ref_P', 'absorption_coefficient_ref_T', 'bnd_limits_gpt', 'bnd_limits_wavenumber', 'gas_minor', 'gas_names', 'identifier_minor', 'minor_gases_lower', 'minor_gases_upper', 'minor_limits_gpt_lower', 'minor_limits_gpt_upper', 'minor_scales_with_density_lower', 'minor_scales_with_density_upper', 'press_ref', 'press_ref_trop', 'scale_by_complement_lower', 'scale_by_complement_upper', 'scaling_gas_lower', 'scaling_gas_upper', 'temp_ref', 'vmr_ref', 'kminor_lower', 'kminor_start_lower', 'kminor_upper', 'kminor_start_upper', 'optimal_angle_fit']


The structure of the file above is different from the structure of the file containing the trained model:

In [15]:
PATH = "/home/x/git/radnn/ukk23test01/neural/data/"
FILE = "lw-g128-210809_both_72_72_test01.nc"
print(list(nc.Dataset(PATH+FILE).variables))

['nn_dimsize', 'nn_activation', 'nn_inputs', 'nn_input_coeffs_max', 'nn_input_coeffs_min', 'nn_activation_char', 'nn_inputs_char', 'nn_weights_1', 'nn_bias_1', 'nn_weights_2', 'nn_bias_2', 'nn_weights_3', 'nn_bias_3', 'nn_output_coeffs_mean', 'nn_output_coeffs_std']


## RFMIP-CLEAR-SKY - run manually without NN

In [16]:
! ./rrtmgp_rfmip_lw \
        8 \
        data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
        ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
        1 \
        1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)]  input_output file]
 OR:  rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index] [physics_index] [NN_lw_abs_file] [NN_lw_planck_file]
 input file:data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                
 nexp:          18 ncol:         100 nlay:          60 block_size:           8
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky LW computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.559000015    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.235573    
 mean of flux_up is:   301.596527    
 ---------------

Full data path:

- /home/x/git/radnn/ukk23test01/rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc
- /home/x/git/radnn/ukk23test01/examples/rfmip-clear-sky/data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc

In [17]:
! ./rrtmgp_rfmip_sw \
        8 \
        data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
        ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
        1

 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)]
 OR:  rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index] [NN_sw_abs_file] [NN_sw_ray_file]
 input file:data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                
 nexp:          18 ncol:         100 nlay:          60
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky SW computations, using lookup-table as RRTMGP kernel
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum toa flux   1360.99963    
 sum t

## Run RRTMGP on the RFMIP off-line test cases

- Run the RFMIP example programs that computes fluxes from netCDF *Garand atmosphere* files.
- Code should be run in the `rfmip_dir` directory.

In [18]:
import os, subprocess

In [19]:
rte_rrtmgp_dir = os.path.join("..", "..")
rfmip_dir = "."
conds_file = (
    "data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
)
lw_gas_coeffs_file = os.path.join(
    rte_rrtmgp_dir,
    "rrtmgp",
    "data",
    "rrtmgp-data-lw-g256-2018-12-04.nc",
)
sw_gas_coeffs_file = os.path.join(
    rte_rrtmgp_dir,
    "rrtmgp",
    "data",
    "rrtmgp-data-sw-g224-2018-12-04.nc",
)
rfmip_lw_exe_name = "./rrtmgp_rfmip_lw"
rfmip_sw_exe_name = "./rrtmgp_rfmip_sw"
block_size_str = "   8"

Arguments are block size, input conditions, coefficient files, forcing index, physics index:

In [20]:
print("Running "
    + rfmip_lw_exe_name
    + block_size_str + " "
    + conds_file + " "
    + lw_gas_coeffs_file
)

Running ./rrtmgp_rfmip_lw   8 data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc


In [21]:
subprocess.run(
    [rfmip_lw_exe_name, block_size_str, conds_file, lw_gas_coeffs_file]
)
subprocess.run(
    [rfmip_sw_exe_name, block_size_str, conds_file, sw_gas_coeffs_file]
)

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)]  input_output file]
 OR:  rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index] [physics_index] [NN_lw_abs_file] [NN_lw_planck_file]
 input file:data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                
 nexp:          18 ncol:         100 nlay:          60 block_size:           8
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky LW computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.540000021    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.235573    
 mean of flux_up is:   301.596527    
 ---------------

CompletedProcess(args=['./rrtmgp_rfmip_sw', '   8', 'data/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc', '../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc'], returncode=0)